In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# all_data

In [ ]:
from neocov.read_data import *
from neocov.preproc import *
from neocov.type_emb import *
from neocov.communities import *

In [ ]:
from gensim.models import Word2Vec
import pandas as pd
pd.set_option('display.max_rows', 100)
from pathlib import Path

In [ ]:
DATA_DIR = '../data/'
COMMENTS_DIAC_DIR = f'{DATA_DIR}comments/by_date/'
OUT_DIR = '../out/'

# NeoCov

> Semantic change and social semantic variation of Covid-related English neologisms on Reddit.

## Semantic change

In [ ]:
YEAR = '2020'

### Read data

#### Get file paths

In [ ]:
comment_paths_year = get_comments_paths_year(COMMENTS_DIAC_DIR, YEAR)

#### Read comments

In [ ]:
%%time
comments = read_comm_csvs(comment_paths_year)

In [ ]:
comments

### Pre-process comments

In [ ]:
%%time
comments_clean = clean_comments(comments)

In [ ]:
docs = comments_clean['body'].to_list()

In [ ]:
import pickle

In [ ]:
with open(f'{OUT_DIR}docs_clean/diac_{YEAR}.pickle', 'wb') as fp:
    pickle.dump(docs, fp)

In [ ]:
with open(f'{OUT_DIR}docs_clean/diac_{YEAR}.pickle', 'rb') as fp:
    docs = pickle.load(fp)

### Train models

#### Create corpus

In [ ]:
corpus = Corpus(docs)

#### Train model

In [ ]:
%%time
model = train_model(corpus, EPOCHS=20)

In [ ]:
len(model.wv.key_to_index)

#### Save model

In [ ]:
model.save(f'{OUT_DIR}models/{YEAR}_ep-20.model')

### Load models

In [ ]:
model_2019 = Word2Vec.load(f'{OUT_DIR}models/2019_ep-20.model')

In [ ]:
model_2020 = Word2Vec.load(f'{OUT_DIR}models/2020_ep-20.model')

### Align models

In [ ]:
model_2019_vocab = len(model_2019.wv.key_to_index)
model_2020_vocab = len(model_2020.wv.key_to_index)

In [ ]:
smart_procrustes_align_gensim(model_2019, model_2020)

In [ ]:
assert len(model_2019.wv.key_to_index) == len(model_2020.wv.vectors)

In [ ]:
models_vocab = pd.DataFrame(
    columns=['Model', 'Words'],
    data=[
        ['2019', model_2019_vocab],
        ['2020', model_2020_vocab],
        ['intersection', len(model_2019.wv.key_to_index)]
    ],
)

models_vocab

In [ ]:
models_vocab.to_csv(f'{OUT_DIR}models_vocab.csv', index=False)

### Measure distances

In [ ]:
distances = measure_distances(model_2019, model_2020)

TODO: filter by true type frequency; `Gensim`'s type frequency seems incorrect; it probably reflects frequency ranks instead of total counts.

In [ ]:
blacklist_lex = load_blacklist_lex()

k = 500
freq_min = 100

sem_change_cands = (distances\
    .query('freq_1 > @freq_min and freq_2 > @freq_min')
    .query('lex.str.isalpha() == True')
    .query('lex.str.len() > 3')
    .query('lex not in @blacklist_lex')
    .nlargest(k, 'dist_sem')
    .reset_index(drop=True)
)

sem_change_cands

In [ ]:
sem_change_cands_out = (sem_change_cands
    .nlargest(100, 'dist_sem')
    .assign(index_1 = lambda df: df.index + 1)
    .assign(dist_sem = lambda df: df['dist_sem'].round(2))
    .assign(dist_sem = lambda df: df['dist_sem'].apply('{:.2f}'.format))
    .rename({'index_1': '', 'lex': 'Lexeme', 'dist_sem': 'SemDist'}, axis=1)
)

In [ ]:
sem_change_cands_out.to_csv(
        f'{OUT_DIR}sem_change_cands.csv',
        columns=['', 'Lexeme', 'SemDist'],
        index=False
    )

### Inspect nearest neighbours of lexemes

In [ ]:
LEX_NBS = 'ahahahah'

In [ ]:
nbs_model_1, nbs_model_2 = get_nearest_neighbours_models(
    lex=LEX_NBS, 
    freq_min=1,
    model_1=model_2019, 
    model_2=model_2020,
    k=10
)

display(
    nbs_model_1,
    nbs_model_2
)

Not related to Covid:

- sunsetting: > gaming-related meaning in 2020
- childe: > gaming-related proper name in 2020
- megalodon: > gaming-related proper name in 2020
- newf: (derogatory) slang term for people from Newfoundland (Canada)
- chaz: > Capitol Hill Autonomous Zone (CHAZ)
- klee: > computer game character, proper name
- rittenhouse: whiskey brand > proper name, involved in shooting related to BLM protests

Related to Covid:

- cerb: > Canada Emergency Response Benefit for Covid
- vacuo: > medical term, 'vacuum'
- moderna: > vaccine

## Social semantic variation

### Inspect subreddits

#### read comments

In [ ]:
comments_dir_path = Path('../data/comments/lexeme/')

In [ ]:
comments_paths = list(comments_dir_path.glob(f'Covid*.csv'))

In [ ]:
%%time
comments = read_comm_csvs(comments_paths)
comments

TODO: filter comments

- [ ] remove duplicates
- [ ] remove bots

#### get subreddit counts

In [ ]:
subr_counts = get_subr_counts(comments)

In [ ]:
subr_counts_plt = plot_subr_counts(subr_counts, k=20)
subr_counts_plt

In [ ]:
subr_counts_plt.save(f'{OUT_DIR}subr_counts.png', scale_factor=2.0)

### Train models

In [ ]:
COMMENTS_DIR_SUBR = '../data/comments/subr/'

In [ ]:
SUBR = 'conspiracy'

In [ ]:
fpaths = get_comments_paths_subr(COMMENTS_DIR_SUBR, SUBR)

In [ ]:
%%time
comments = read_comm_csvs(fpaths)

In [ ]:
%%time
comments_clean = clean_comments(comments)

In [ ]:
docs = comments_clean['body']

In [ ]:
docs = docs.to_list()

In [ ]:
import pickle

In [ ]:
with open(f'{OUT_DIR}docs_clean/subr_{SUBR}.pickle', 'wb') as fp:
    pickle.dump(docs, fp)

In [ ]:
with open('{OUT_DIR}docs_clean/subr_{SUBR}.pickle', 'rb') as fp:
    docs = pickle.load(fp)

Corpus information

| Subreddit          | Comments  | DateFirst  | DateLast   |
|:-------------------|---------: |:-----------|:-----------|
| LockdownSkepticism |   520,392 | 2020-03-26 | 2020-12-27 |  
| Coronavirus        | 4,121,144 | 2020-01-21 | 2020-12-27 |
| conspiracy         | 3,973,514 | 2020-01-01 | 2020-12-27 |

In [ ]:
corpus = Corpus(docs)

In [ ]:
%%time
model = train_model(corpus)

In [ ]:
len(model.wv.key_to_index)

In [ ]:
model.save(f'{OUT_DIR}models/{SUBR}.model')

### Load models

In [ ]:
SUBRS = ['Coronavirus', 'conspiracy']

In [ ]:
model_1 = Word2Vec.load(f'{OUT_DIR}models/{SUBRS[0]}.model')

In [ ]:
model_2 = Word2Vec.load(f'{OUT_DIR}models/{SUBRS[1]}.model')

### Align models

In [ ]:
model_1_vocab = len(model_1.wv.key_to_index)
model_2_vocab = len(model_2.wv.key_to_index)

In [ ]:
smart_procrustes_align_gensim(model_1, model_2)

67181 67181
67181 67181


In [ ]:
assert len(model_1.wv.key_to_index) == len(model_2.wv.vectors)

In [ ]:
models_vocab = pd.DataFrame(
    columns=['Model', 'Words'],
    data=[
        [SUBRS[0], model_1_vocab],
        [SUBRS[1], model_2_vocab],
        ['intersection', len(model_1.wv.key_to_index)]
    ],
)

models_vocab

,Model,Words
0,Coronavirus,94816
1,conspiracy,112599
2,intersection,67181


In [ ]:
models_vocab.to_csv(f'{OUT_DIR}models_subrs_vocab.csv', index=False)

### Measure distances

In [ ]:
distances = measure_distances(model_1, model_2)

#### words that differ the most between both communities

In [ ]:
blacklist_lex = load_blacklist_lex()

k = 20
freq_min = 100

sem_change_cands = (distances\
    .query('freq_1 > @freq_min and freq_2 > @freq_min')
    .query('lex.str.isalpha() == True')
    .query('lex.str.len() > 3')
    .query('lex not in @blacklist_lex')
    .nlargest(k, 'dist_sem')
    .reset_index(drop=True)
)

sem_change_cands

,lex,dist_sem,freq_1,freq_2
0,soliciting,0.989504,2233,2474
1,resubmit,0.944866,1763,1928
2,waiters,0.929376,149,206
3,curiously,0.928779,118,164
4,subsequently,0.924492,12174,11956
5,anons,0.920569,260,351
6,blacklivesmatter,0.918460,107,150
7,redpill,0.914164,119,165
8,derek,0.913568,211,287
9,borderline,0.892894,21891,20974


In [ ]:
sem_change_cands_out = (sem_change_cands
    .nlargest(100, 'dist_sem')
    .assign(index_1 = lambda df: df.index + 1)
    .assign(dist_sem = lambda df: df['dist_sem'].round(2))
    .assign(dist_sem = lambda df: df['dist_sem'].apply('{:.2f}'.format))
    .rename({'index_1': '', 'lex': 'Lexeme', 'dist_sem': 'SemDist'}, axis=1)
)
sem_change_cands_out.to_csv(
        f'{OUT_DIR}sem_var_soc_cands.csv',
        columns=['', 'Lexeme', 'SemDist'],
        index=False
    )

#### nearest neighbours for target lexemes in both communities

In [ ]:
LEX_NBS = 'lockdown'

In [ ]:
nbs_model_1, nbs_model_2 = get_nearest_neighbours_models(
    lex=LEX_NBS, 
    freq_min=50,
    model_1=model_1, 
    model_2=model_2,
    k=10
)

display(
    nbs_model_1,
    nbs_model_2
)

,model,lex,similarity,freq
0,1,shutdown,0.844873,5413
1,1,lockdowns,0.768522,16637
2,1,shutdowns,0.661069,1632
3,1,curfew,0.613175,1079
4,1,quarantine,0.600933,26502
5,1,restrictions,0.581443,16482
6,1,quarantines,0.568791,1634
7,1,quarentine,0.540457,255
8,1,curfews,0.536410,456
9,1,containment,0.515410,2526


,model,lex,similarity,freq
29861,2,lockdowns,0.794638,15881
29862,2,shutdown,0.726183,5529
29863,2,quarantine,0.723483,24948
29864,2,shutdowns,0.702284,1805
29865,2,quarantines,0.622162,1806
29866,2,curfew,0.595864,1236
29867,2,restrictions,0.583930,15817
29868,2,curfews,0.581970,577
29869,2,pandemic,0.580793,67364
29870,2,quarentine,0.528733,346


#### embeddings projection

In [ ]:
from scipy import spatial
import altair as alt
import numpy as np

In [ ]:
models = []
models.append({'subreddit': SUBRS[0], 'model': model_1})
models.append({'subreddit': SUBRS[1], 'model': model_2})

In [ ]:
def get_pole_avg(model, lex, k):
	vecs = []
	vecs.append(model.wv[lex])
	for closest_word, similarity in model.wv.most_similar(positive=lex, topn=k):
		vecs.append(model.wv[closest_word])
	pole_avg = np.mean(vecs, axis=0)
	return pole_avg

In [ ]:
def make_sem_axis(model, pole_word_1: str, pole_word_2: str):
	pole_1_vec = model_1.wv.get_vector(pole_1)
	pole_2_vec = model_1.wv.get_vector(pole_2)
	sem_axis = pole_1_vec - pole_2_vec
	return sem_axis

In [ ]:
def make_sem_axis_avg(model, pole_word_1: str, pole_word_2: str):
	pole_1_vec = model_1.wv.get_vector(pole_1)
	pole_2_vec = model_1.wv.get_vector(pole_2)
	pole_1_avg = get_pole_avg(model_1, pole_word_1, k=10)
	pole_2_avg = get_pole_avg(model_1, pole_word_2, k=10)
	sem_axis = pole_1_avg - pole_2_avg
	return sem_axis

In [ ]:
def get_axis_sim(lex: str, pole_word_1: str, pole_word_2: str, model):
	sem_axis = make_sem_axis_avg(model, pole_word_1, pole_word_2)
	lex_vec = model.wv.get_vector(lex)
	sim_cos = 1 - spatial.distance.cosine(lex_vec, sem_axis)
	return sim_cos

In [ ]:
lex = 'lockdown'
pole_1 = 'good'
pole_2 = 'bad'

In [ ]:
lexs = [
	'lockdown', 'lockdowns', 
	'shutdown', 'shutdowns', 
	'vaccine', 'vaccines', 
	'mask', 'masks',
	'order', 'police', 'science', 'mandate',
	'thing', 'tree', 
	'give', 'take'
	# 'happy', 'sad',
	]

In [ ]:
sims = []
for lex in lexs:
	for model in models:
		sim = {}
		sim['subreddit'] = model['subreddit']
		sim['lex'] = lex
		sim['sim'] = get_axis_sim(lex, pole_1, pole_2, model['model'])
		sims.append(sim)

In [ ]:
sims_df = pd.DataFrame(sims)

alt.Chart(sims_df).mark_line(point=True).encode(
	x='sim',
	y=alt.Y('lex', sort=None),
	color='subreddit'
)


alt.Chart(...)

#### biggest discrepancies in nearest neighbours for target lexemes

In [ ]:
nbs_model_1, nbs_model_2 = get_nearest_neighbours_models(
    lex=LEX, 
    freq_min=150,
    model_1=model_1, 
    model_2=model_2,
    k=100_000
)

In [ ]:
nbs_diffs = pd.merge(
    nbs_model_1, nbs_model_2, 
    on='lex',
    suffixes = ('_1', '_2')
)

In [ ]:
nbs_diffs = nbs_diffs\
    .assign(sim_diff = abs(nbs_diffs['similarity_1'] - nbs_diffs['similarity_2']))\
    .sort_values('sim_diff', ascending=False)\
    .reset_index(drop=True)\
    .query('lex.str.len() >= 4')

In [ ]:
topn = 10

subr_1_nbs = nbs_diffs\
    .query('similarity_1 > similarity_2')\
    .nlargest(topn, 'sim_diff')

subr_2_nbs = nbs_diffs\
    .query('similarity_2 > similarity_1')\
    .nlargest(topn, 'sim_diff')

display(subr_1_nbs, subr_2_nbs)